In [105]:
import pandas as pd

In [106]:
df = pd.read_csv("scrapper/UK_raw_data.csv")
df.head()

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors
0,Data Science,£24K - £38K (Glassdoor est.),Love using technology to solve complex problem...,4.1,MThree Consulting\n4.1,"London, England",-1,501 to 1000 Employees,2010,Company - Private,IT Services,Information Technology,Unknown / Non-Applicable,-1
1,Data Science Graduate Scheme,£24K - £38K (Glassdoor est.),"Salary and Benefits: £24,500 + pension + 25 da...",3.6,Yorkshire Water\n3.6,"Bradford, Yorkshire and the Humber, England",-1,1001 to 5000 Employees,1973,Company - Private,Utilities,"Oil, Gas, Energy & Utilities",$500 million to $1 billion (USD),-1
2,Data Engineer,£24K - £38K (Glassdoor est.),Job Description\n\nData Engineer\n\nAt Citadel...,4.3,Citadel\n4.3,"London, England",-1,1001 to 5000 Employees,1990,Company - Private,-1,-1,$50 to $100 million (USD),-1
3,Data Scientist (Investigations),£24K - £38K (Glassdoor est.),Who are we?\n\nHi! �� We are Ravelin! We're a ...,4.8,Ravelin\n4.8,"London, England",-1,51 to 200 Employees,2014,Company - Private,Enterprise Software & Network Solutions,Information Technology,$5 to $10 million (USD),-1
4,Data/Marketing Analyst,£24K - £38K (Glassdoor est.),Data / Marketing Analyst\n\nThe Marketing team...,4.6,gohenry Ltd\n4.6,"London, England",-1,51 to 200 Employees,2012,Company - Private,Investment Banking & Asset Management,Finance,$10 to $25 million (USD),-1


In [107]:
df.shape
df.drop_duplicates(inplace=True)

In [108]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 880 entries, 0 to 932
Data columns (total 14 columns):
Job Title            880 non-null object
Salary Estimate      880 non-null object
Job Description      880 non-null object
Rating               880 non-null float64
Company Name         880 non-null object
Location             880 non-null object
Headquarters         880 non-null int64
Size                 880 non-null object
Founded              880 non-null int64
Type of ownership    880 non-null object
Industry             880 non-null object
Sector               880 non-null object
Revenue              880 non-null object
Competitors          880 non-null int64
dtypes: float64(1), int64(3), object(10)
memory usage: 103.1+ KB


#### 1. Salary Estimate Parsing

In [109]:
# df['hourly'] = df['Salary Estimate'].apply(lambda x : 1 if 'per hour' in x.lower() else 0)
# df['hourly'].value_counts() --> NO Hourly Salaries in this dateset
# df['employer provided'] = df['Salary Estimate'].apply(lambda x: 1 if 'employer provided salary:' in x.lower() else 0)
# df['employer provided']. value_counts() --> NO such data

df = df[df['Salary Estimate'] != '-1'] 
salary = df['Salary Estimate'].apply(lambda x:x.split("(")[0])
remove_symbols = salary.apply(lambda x: x.replace('K','').replace('£',''))
print(salary[0])
print(remove_symbols[0])

£24K - £38K 
24 - 38 


In [110]:
df['min_salary'] = remove_symbols.apply(lambda x: int(x.split('-')[0]))
print(df['min_salary'][0])
df['max_salary'] = remove_symbols.apply(lambda x: int(x.split('-')[1]))
print(df['max_salary'][0])
df['avg_salary'] = (df['min_salary'] + df['max_salary']) / 2

24
38


#### 2. Company Name

In [111]:
df['Company Name'] = df.apply(lambda x: x['Company Name'] if x['Rating']<0 else x['Company Name'].split('\n')[0], axis=1)
print(df['Company Name'][0:2])

0    MThree Consulting
1      Yorkshire Water
Name: Company Name, dtype: object


In [112]:
df.drop(['Headquarters', 'Competitors'], axis=1, inplace=True)

In [113]:
df.head(1)

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Size,Founded,Type of ownership,Industry,Sector,Revenue,min_salary,max_salary,avg_salary
0,Data Science,£24K - £38K (Glassdoor est.),Love using technology to solve complex problem...,4.1,MThree Consulting,"London, England",501 to 1000 Employees,2010,Company - Private,IT Services,Information Technology,Unknown / Non-Applicable,24,38,31.0


#### 3. Job Description

In [114]:
df['python'] = df['Job Description'].apply(lambda x: 1 if 'python' in x.lower() else 0)
print(df['python'].value_counts())

df['sql'] = df['Job Description'].apply(lambda x: 1 if 'sql' in x.lower() else 0)
print(df['sql'].value_counts())

df['r_studio'] = df['Job Description'].apply(lambda x: 1 if 'r studio' in x.lower() or 'r-studio' in x.lower() else 0)
print(df['r_studio'].value_counts())

df['spark'] = df['Job Description'].apply(lambda x: 1 if 'spark' in x.lower() else 0)
print(df['spark'].value_counts())

df['aws'] = df['Job Description'].apply(lambda x: 1 if 'aws' in x.lower() else 0)
print(df['aws'].value_counts())

df['excel'] = df['Job Description'].apply(lambda x: 1 if 'excel' in x.lower() else 0)
df.excel.value_counts()

0    693
1    187
Name: python, dtype: int64
0    694
1    186
Name: sql, dtype: int64
0    879
1      1
Name: r_studio, dtype: int64
0    839
1     41
Name: spark, dtype: int64
0    790
1     90
Name: aws, dtype: int64


1    548
0    332
Name: excel, dtype: int64

#### 4. Founded --> age

In [115]:
df['age'] = df['Founded'].apply(lambda x: x if x<1 else 2021-x)
df['age'][0:5]

0    11
1    48
2    31
3     7
4     9
Name: age, dtype: int64

#### 5. Size --> employees

In [116]:
df['Size'].value_counts()

10000+ Employees           279
1001 to 5000 Employees     149
51 to 200 Employees        122
1 to 50 Employees           97
-1                          70
5001 to 10000 Employees     60
201 to 500 Employees        45
501 to 1000 Employees       35
Unknown                     23
Name: Size, dtype: int64

#### 6. Job Title

In [119]:
def title_simplifier(title):
    if 'data scientist' in title.lower(): return 'data scientist'
    elif 'data engineer' in title.lower(): return 'data engineer'
    elif 'analyst' in title.lower(): return 'analyst'
    elif 'machine learning' in title.lower(): return 'mle'
    elif 'manager' in title.lower(): return 'manager'
    elif 'director' in title.lower(): return 'director'
    elif 'consultant' in title.lower(): return 'consultant'
    elif 'researcher' in title.lower(): return 'ai researcher'
    else: return 'na'

def seniority(title):
    if 'sr' in title.lower() or 'senior' in title.lower() or 'sr' in title.lower() or 'lead' in title.lower() or 'principal' in title.lower():
        return 'senior'
    elif 'jr' in title.lower() or 'jr.' in title.lower() or 'junior' in title.lower():
        return 'junior'
    else:
        return 'na'
    

In [120]:
df['job_title'] = df['Job Title'].apply(title_simplifier)
print(df['job_title'].value_counts())

df['seniority'] = df['Job Title'].apply(seniority)
print(df['seniority'].value_counts())

na                425
manager           172
director           88
analyst            82
data scientist     59
ai researcher      22
consultant         19
data engineer      12
mle                 1
Name: job_title, dtype: int64
na        676
senior    204
Name: seniority, dtype: int64
